# CS 4501 - Project 3 Part 2: Leduc Poker + RL Algorithms

Rishi Shah, Yash Kolli, Tarun Donipati

In this project, our team explores the applications of reinforcement learning to Leduc poker, a simplified variation of poker involving face card combinations. We first initialize two objects that are key to our game — balance and card. Each player starts with a balance of 100.

In [52]:
import random

class HumanPlayer:
    def __init__(self, balance, card, wins):
        self.balance = balance
        self.card = card
        self.wins = wins

class AIPlayer:
    def __init__(self, balance, card, wins, epsilon, alpha, state):
        self.balance = balance
        self.card = card
        self.wins = wins
        self.epsilon = epsilon
        self.alpha = alpha
        self.state = state

In [53]:
state = {
            'KK0': .5, 'KK1': .5, 'KK01': .5, 'KK10': .5, 'KK11': .5, 'KK00': .5, 'KK010': .5, 'KK011': .5,
            'QQ0': .5, 'QQ1': .5, 'QQ01': .5, 'QQ10': .5, 'QQ11': .5, 'QQ00': .5, 'QQ010': .5, 'QQ011': .5,
            'JJ0': .5, 'JJ1': .5, 'JJ01': .5, 'JJ10': .5, 'JJ11': .5, 'JJ00': .5, 'JJ010': .5, 'JJ011': .5,
            'KQ0': .5, 'KQ1': .5, 'KQ01': .5, 'KQ10': .5, 'KQ11': .5, 'KQ00': .5, 'KQ010': .5, 'KQ011': .5,
            'QK0': .5, 'QK1': .5, 'QK01': .5, 'QK10': .5, 'QK11': .5, 'QK00': .5, 'QK010': .5, 'QK011': .5,
            'KJ0': .5, 'KJ1': .5, 'KJ01': .5, 'KJ10': .5, 'KJ11': .5, 'KJ00': .5, 'KJ010': .5, 'KJ011': .5,
            'JK0': .5, 'JK1': .5, 'JK01': .5, 'JK10': .5, 'JK11': .5, 'JK00': .5, 'JK010': .5, 'JK011': .5,
            'QJ0': .5, 'QJ1': .5, 'QJ01': .5, 'QJ10': .5, 'QJ11': .5, 'QJ00': .5, 'QJ010': .5, 'QJ011': .5, 
            'JQ0': .5, 'JQ1': .5, 'JQ01': .5, 'JQ10': .5, 'JQ11': .5, 'JQ00': .5, 'JQ010': .5, 'JQ011': .5,
        }

In the cell below, we lay out the rules of the game. Leduc poker is a two-player game where each player receives a face card (K, Q, J), and a community card of some face is chosen at random. The objective of the game is to have your card match with the community card to create the pairing of the highest value.

The pair values progress as follows — KK, QQ, JJ, KQ, KJ, QJ — where KK represents the highest value while QJ represents the lowest value. We thus order our values ordinally, where KK = 5, QQ = 4, JJ = 3, KQ = 2, KJ = 1, and QJ = 0. 

In [54]:
def showdownValue(card, community):
    
    ranks = {
        'KK': 1,
        'QQ': 2,
        'JJ': 3,
        'KQ': 4, 'QK': 4,
        'KJ': 5, 'JK': 5,
        'QJ': 6, 'JQ': 6
    }

    hand = card+community
    return ranks[hand]

The following cell is used to normalize winning probablities for pairs of actions.

In [55]:
def normalize(bet, check):
    if bet < 0:
        return 0, 1
    
    if check < 0:
        return 1, 0
    
    #print(bet, check)
    
    if bet + check > 0:
        bet /= (bet + check)
        check /= (bet + check)
        
    elif bet + check == 0:
        bet = 0.5
        check = 0.5
        
    #print(bet, check)
    
    return bet, check

The game progresses as follows. First, Player 1 and Player 2 are each given a random face card (K, Q, or J). Simultaneously, a community card is chosen at random. When each player decides to partake in the game, they transfer 1 unit of their balance to the pot. Each player can only see their card and the community card, and must decide whether to increase the pot by betting more or to hold their balance by checking or folding.

A player has three moves to make in each game: raise, call, or fold. A rational player raises when they predict having a high chance of winning. For example, if I saw that I have a KK pair, I would raise the pot as much as possible, as that would result in a higher gain for me. Raising is the only attacking move that a player can make in poker. 

Call and fold are both reactionary moves to a raise. A rational player would call a bet when they believe that they have a good chance of winning. For example, if I have a KK pair and my opponent raised the pot, I would call the pot to stay in the game. A player would fold when they believe that they don't have a high chance of winning. For example, if I have a QJ pair and my opponent raises, I would fold to make sure that I don't lose more money than what I already contributed to the pot.

Another element to this game is that the player can choose how much of their balance they commit to raising the pot. If I predict having a high chance of winning, I can raise the pot by a lot. However, if I only predict having a small chance, I would only raise by a small amount.

At the same time, if the opposing player sees a high raise from me, they can detect me signaling my winning probability. As such, a raise that is too high will prompt the opposing player to fold so that they don't lose their winnings.

Thus, a rational player must optimize their raise such that it balances high raise with high probability of call.

In [56]:
def LeducPokerRandomChoices(Player1, Player2):
    
    # initialize cards
    cards = ['J', 'J', 'Q', 'Q', 'K', 'K']
    
    # initialize money pot
    pot = 0
    
    # randomly assign cards
    s = random.sample(cards, 3)

    Player1.card = s[0]
    Player2.card = s[1]
    community_card = s[2]
    
    
    p1 = [Player1.card + community_card]
    p2 = [Player2.card + community_card]
      
    
    # add initial ante to the pot
    Player1.balance -= 1
    Player2.balance -= 1
    pot += 2
    
    
    # first action by Player 1
    if Player1.state[p1[0] + '1'] > Player1.state[p1[0] + '0']:
        if random.random() > Player1.epsilon:
            player1_action1 = 1
        else:
            player1_action1 = 0
    
    elif Player1.state[p1[0] + '1'] < Player1.state[p1[0] + '0']:
        if random.random() > Player1.epsilon:
            player1_action1 = 0
        else:
            player1_action1 = 1
    
    else:
        player1_action1 = round(random.random())
    
    
    p1[0] += str(player1_action1)
    p2[0] += str(player1_action1)
    
    
    # execute if Player 1 checks
    if player1_action1 == 0:
        
        # first action by Player 2
        if Player2.state[p2[0] + '1'] > Player2.state[p2[0] + '0']:
            if random.random() > Player2.epsilon:
                player2_action1 = 1
            else:
                player2_action1 = 0
    
        elif Player2.state[p2[0] + '1'] < Player2.state[p2[0] + '0']:
            if random.random() > Player2.epsilon:
                player2_action1 = 0
            else:
                player2_action1 = 1
    
        else:
            player2_action1 = round(random.random())
            
        p1.append(p1[0] + str(player2_action1))
        p2.append(p2[0] + str(player2_action1))
        
        # execute if Player 2 checks
        if player2_action1 == 0:
    
            # showdown for the pot
        
            # Player 1 wins
            if showdownValue(Player1.card, community_card) > showdownValue(Player2.card, community_card):
                Player1.balance += pot
                pot = 0
                Player1.wins += 1
                
                for i in range(len(p1)):
                    Player1.state[p1[i]] += Player1.alpha
                    Player2.state[p2[i]] -=  Player2.alpha
                    
                    
                    if p1[i][-1] == '1':
                        Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'] = normalize(Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'])
                        
                    else:
                        Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]] = normalize(Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]])
                        
                    if p2[i][-1] == '1':
                        Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'] = normalize(Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'])
                        
                    else:
                        Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]] = normalize(Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]])
                
            
            # Player 2 wins            
            elif showdownValue(Player2.card, community_card) > showdownValue(Player1.card, community_card):
                Player2.balance += pot
                pot = 0
                Player2.wins += 1
                
                for i in range(len(p1)):
                    Player2.state[p2[i]] += Player2.alpha
                    Player1.state[p1[i]] -= Player1.alpha
                    
                    if p1[i][-1] == '1':
                        Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'] = normalize(Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'])
                        
                    else:
                        Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]] = normalize(Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]])
                        
                    if p2[i][-1] == '1':
                        Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'] = normalize(Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'])
                        
                    else:
                        Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]] = normalize(Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]])
                
            # Tie Game
            else:
                Player1.balance += pot/2
                Player2.balance += pot/2
                pot = 0
        
        # execute if Player 2 bets
        if player2_action1 == 1:
            
            # add to the pot from Player 2
            Player2.balance -= 1
            pot += 1
            
            # second action by Player 1
            if Player1.state[p1[1] + '1'] > Player1.state[p1[1] + '0']:
                if random.random() > Player1.epsilon:
                    player1_action2 = 1
                else:
                    player1_action2 = 0
    
            elif Player1.state[p1[1] + '1'] < Player1.state[p1[1] + '0']:
                if random.random() > Player1.epsilon:
                    player1_action2 = 0
                else:
                    player1_action2 = 1
    
            else:
                player1_action2 = round(random.random())
                
            p1.append(p1[1] + str(player1_action2))
            p2.append(p2[1] + str(player1_action2))
            
            # execute if Player 1 folds
            if player1_action2 == 0:
                Player2.balance += pot
                pot = 0
                Player2.wins += 1
                for i in range(len(p1)):
                    Player2.state[p2[i]] += Player2.alpha
                    Player1.state[p1[i]] -= Player1.alpha
                    
                    
                    if p1[i][-1] == '1':
                        Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'] = normalize(Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'])
                        
                    else:
                        Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]] = normalize(Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]])
                        
                    if p2[i][-1] == '1':
                        Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'] = normalize(Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'])
                        
                    else:
                        Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]] = normalize(Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]])
                
        
            # execute if Player 1 calls
            if player1_action2 == 1:
            
                # add to the pot from Player 1
                Player1.balance -= 1
                pot += 1
            
                # showdown for the pot
                
                # Player 1 wins
                if showdownValue(Player1.card, community_card) > showdownValue(Player2.card, community_card):
                    Player1.balance += pot
                    pot = 0
                    Player1.wins += 1
                    for i in range(len(p1)):
                        Player1.state[p1[i]] += Player1.alpha
                        Player2.state[p2[i]] -= Player2.alpha
                        
                        
                        if p1[i][-1] == '1':
                            Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'] = normalize(Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'])
                        
                        else:
                            Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]] = normalize(Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]])
                        
                        if p2[i][-1] == '1':
                            Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'] = normalize(Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'])
                        
                        else:
                            Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]] = normalize(Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]])
                    
                
            
                # Player 2 wins            
                elif showdownValue(Player2.card, community_card) > showdownValue(Player1.card, community_card):
                    Player2.balance += pot
                    pot = 0
                    Player2.wins += 1
                    for i in range(len(p1)):
                        Player2.state[p2[i]] += Player2.alpha
                        Player1.state[p1[i]] -= Player1.alpha
                        
                        
                        if p1[i][-1] == '1':
                            Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'] = normalize(Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'])
                        
                        else:
                            Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]] = normalize(Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]])
                        
                        if p2[i][-1] == '1':
                            Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'] = normalize(Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'])
                        
                        else:
                            Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]] = normalize(Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]])
                    
                    
                
                # Tie Game
                else:
                    Player1.balance += pot/2
                    Player2.balance += pot/2
                    pot = 0
            
    
    # execute if Player 1 bets
    if player1_action1 == 1:
        
        # add to the pot from Player 1
        Player1.balance -= 1
        pot += 1
        
        # first action by Player 2
        if Player2.state[p2[0] + '1'] > Player2.state[p2[0] + '0']:
            if random.random() > Player2.epsilon:
                player2_action1 = 1
            else:
                player2_action1 = 0
    
        elif Player2.state[p2[0] + '1'] < Player2.state[p2[0] + '0']:
            if random.random() > Player2.epsilon:
                player2_action1 = 0
            else:
                player2_action1 = 1
    
        else:
            player2_action1 = round(random.random())
            
        p1.append(p1[0] + str(player2_action1))
        p2.append(p2[0] + str(player2_action1))
        
        # execute if Player 2 folds
        if player2_action1 == 0:
            Player1.balance += pot
            pot = 0
            Player1.wins += 1
            for i in range(len(p1)):
                Player1.state[p1[i]] += Player1.alpha
                Player2.state[p2[i]] -= Player2.alpha
                
                
                if p1[i][-1] == '1':
                    Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'] = normalize(Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'])
                        
                else:
                    Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]] = normalize(Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]])
                        
                if p2[i][-1] == '1':
                    Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'] = normalize(Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'])
                        
                else:
                    Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]] = normalize(Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]])
            
        
        # execute if Player 2 calls
        if player2_action1 == 1:
            
            # add to the pot from Player 2
            Player2.balance -= 1
            pot += 1
            
            # showdown for the pot
                
            # Player 1 wins
            if showdownValue(Player1.card, community_card) > showdownValue(Player2.card, community_card):
                Player1.balance += pot
                pot = 0
                Player1.wins += 1
                for i in range(len(p1)):
                    Player1.state[p1[i]] += Player1.alpha
                    Player2.state[p2[i]] -= Player2.alpha
                    
                    if p1[i][-1] == '1':
                        Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'] = normalize(Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'])
                        
                    else:
                        Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]] = normalize(Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]])
                        
                    if p2[i][-1] == '1':
                        Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'] = normalize(Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'])
                        
                    else:
                        Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]] = normalize(Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]])
                
            
            # Player 2 wins            
            elif showdownValue(Player2.card, community_card) > showdownValue(Player1.card, community_card):
                Player2.balance += pot
                pot = 0
                Player2.wins += 1
                for i in range(len(p1)):
                    Player2.state[p2[i]] += Player2.alpha # * (Player2.state[p2[i]] - Player2.state[p2[i][0:-1]])
                    Player1.state[p1[i]] -= Player1.alpha
                    
                    if p1[i][-1] == '1':
                        Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'] = normalize(Player1.state[p1[i]], Player1.state[p1[i][0:-1] + '0'])
                        
                    else:
                        Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]] = normalize(Player1.state[p1[i][0:-1] + '1'], Player1.state[p1[i]])
                        
                    if p2[i][-1] == '1':
                        Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'] = normalize(Player2.state[p2[i]], Player2.state[p2[i][0:-1] + '0'])
                        
                    else:
                        Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]] = normalize(Player2.state[p2[i][0:-1] + '1'], Player2.state[p2[i]])
            
            # Tie Game
            else:
                Player1.balance += pot/2
                Player2.balance += pot/2
                pot = 0

In [57]:
def train(epochs, p1, p2):
    
    for i in range(1, epochs + 1):
        LeducPokerRandomChoices(p1, p2)
        
        '''if i % 2 == 1:
            LeducPokerRandomChoices(Player1, Player2)
        else:
            LeducPokerRandomChoices(Player2, Player1)'''
    
    print(Player1.wins, Player1.balance)
    print(Player2.wins, Player2.balance)
    return (Player1.state, Player2.state)
    # print(Player2.state)
    
def compete(epochs, p1, p2):
    for i in range(1, epochs + 1):
        LeducPokerRandomChoices(p1, p2)
    print(Player1.wins, Player1.balance)
    print(Player2.wins, Player2.balance)

In [58]:
s1 = {
            'KK0': .5, 'KK1': 1, 'KK01': .5, 'KK10': 1, 'KK11': 1, 'KK00': 1, 'KK010': 0, 'KK011': 1,
            'QQ0': .5, 'QQ1': 1, 'QQ01': .5, 'QQ10': 1, 'QQ11': 1, 'QQ00': 1, 'QQ010': 0, 'QQ011': 1,
            'JJ0': .5, 'JJ1': 1, 'JJ01': .5, 'JJ10': 1, 'JJ11': 1, 'JJ00': 1, 'JJ010': 0, 'JJ011': 1,
            'KQ0': .5, 'KQ1': .5, 'KQ01': .5, 'KQ10': 1, 'KQ11': .5, 'KQ00': .5, 'KQ010': 0, 'KQ011': .5,
            'QK0': .5, 'QK1': .5, 'QK01': .5, 'QK10': 1, 'QK11': .5, 'QK00': .5, 'QK010': 0, 'QK011': .5,
            'KJ0': .5, 'KJ1': .5, 'KJ01': .5, 'KJ10': 1, 'KJ11': .5, 'KJ00': .5, 'KJ010': 0, 'KJ011': .5,
            'JK0': .5, 'JK1': .5, 'JK01': .5, 'JK10': 1, 'JK11': .5, 'JK00': .5, 'JK010': 0, 'JK011': .5,
            'QJ0': .5, 'QJ1': .5, 'QJ01': .5, 'QJ10': 1, 'QJ11': .5, 'QJ00': .5, 'QJ010': 0, 'QJ011': .5, 
            'JQ0': .5, 'JQ1': .5, 'JQ01': .5, 'JQ10': 1, 'JQ11': .5, 'JQ00': .5, 'JQ010': 0, 'JQ011': .5,
        }

s2 = {
            'KK0': .5, 'KK1': .5, 'KK01': 1, 'KK10': 0, 'KK11': 1, 'KK00': 1, 'KK010': 1, 'KK011': 1,
            'QQ0': .5, 'QQ1': .5, 'QQ01': 1, 'QQ10': 0, 'QQ11': 1, 'QQ00': 1, 'QQ010': 1, 'QQ011': 1,
            'JJ0': .5, 'JJ1': .5, 'JJ01': 1, 'JJ10': 0, 'JJ11': 1, 'JJ00': 1, 'JJ010': 1, 'JJ011': 1,
            'KQ0': .5, 'KQ1': .5, 'KQ01': .5, 'KQ10': 0, 'KQ11': .5, 'KQ00': .5, 'KQ010': 1, 'KQ011': .5,
            'QK0': .5, 'QK1': .5, 'QK01': .5, 'QK10': 0, 'QK11': .5, 'QK00': .5, 'QK010': 1, 'QK011': .5,
            'KJ0': .5, 'KJ1': .5, 'KJ01': .5, 'KJ10': 0, 'KJ11': .5, 'KJ00': .5, 'KJ010': 1, 'KJ011': .5,
            'JK0': .5, 'JK1': .5, 'JK01': .5, 'JK10': 0, 'JK11': .5, 'JK00': .5, 'JK010': 1, 'JK011': .5,
            'QJ0': .5, 'QJ1': .5, 'QJ01': .5, 'QJ10': 0, 'QJ11': .5, 'QJ00': .5, 'QJ010': 1, 'QJ011': .5, 
            'JQ0': .5, 'JQ1': .5, 'JQ01': .5, 'JQ10': 0, 'JQ11': .5, 'JQ00': .5, 'JQ010': 1, 'JQ011': .5,
        }





Player1 = AIPlayer(1000, '', 0, 0.1, 0.005, s1)
Player2 = AIPlayer(1000, '', 0, 0.1, 0.005, s2)

s1, s2 = train(100000, Player1, Player2)

Player1 = AIPlayer(1000, '', 0, 0, 0, s1)
Player2 = AIPlayer(1000, '', 0, 0, 0, s2)

compete(100000, Player1, Player2)


41955 -2625.0
39753 4625.0
40263 -8291.0
39844 10291.0


In [59]:
def LeducPokerPvC(Player1, Player2):
    
    # initialize cards
    cards = ['J', 'J', 'Q', 'Q', 'K', 'K']
    
    # initialize money pot
    pot = 0
    
    s = random.sample(cards, 3)

    Player1.card = s[0]
    Player2.card = s[1]
    community_card = s[2]
    
    p2 = [Player2.card + community_card]
    
    print("Player 1 Card:", Player1.card)
    print("Player 1 Balance:", Player1.balance)
    print()
    print("Player 2 Card not shown")
    print("Player 2 Balance:", Player2.balance)
    print()
    print("Community Card:", community_card)
    print()
    print()
    
    print("Initial bets of 1 made")
    print()
    
    # add initial ante to the pot
    Player1.balance -= 1
    Player2.balance -= 1
    pot += 2
    
    # first action by Player 1
    print("Player 1 Balance", Player1.balance)
    print("Player 1, do you choose to check or bet")
    print("Press 0 for check, 1 to bet")
    print()
    player1_action1 = int(input())
    
    p2[0] += str(player1_action1)
    
    
    # execute if Player 1 checks
    if player1_action1 == 0:
        
        # first action by Player 2
        print("Player 2 Balance", Player2.balance)
        print()
        if Player2.state[p2[0] + '1'] > Player2.state[p2[0] + '0']:
            player2_action1 = 1
        elif Player2.state[p2[0] + '1'] < Player2.state[p2[0] + '0']:
            player2_action1 = 0
        else:
            player2_action1 = round(random.random())
            
        p2.append(p2[0] + str(player2_action1))
        
        if player2_action1 == 1:
            print("Player 2 bets")
        else:
            print("Player 2 checks")
        
        # execute if Player 2 checks
        if player2_action1 == 0:
    
            # showdown for the pot
        
            # Player 1 wins
            if showdownValue(Player1.card, community_card) > showdownValue(Player2.card, community_card):
                Player1.balance += pot
                print("Player 1 Wins")
                print("Player 1 Card:", Player1.card)
                print("Player 1 Balance:", Player1.balance)
                print()
                print("Player 2 Card:", Player2.card)
                print("Player 2 Balance:", Player2.balance)
                print()
                pot = 0
                
            
            # Player 2 wins            
            elif showdownValue(Player2.card, community_card) > showdownValue(Player1.card, community_card):
                Player2.balance += pot
                print("Player 2 Wins")
                print("Player 1 Card:", Player1.card)
                print("Player 1 Balance:", Player1.balance)
                print()
                print("Player 2 Card:", Player2.card)
                print("Player 2 Balance:", Player2.balance)
                print()
                pot = 0
                
            # Tie Game
            else:
                Player1.balance += pot/2
                Player2.balance += pot/2
                pot = 0
                return "Draw"
        
        # execute if Player 2 bets
        if player2_action1 == 1:
            
            # add to the pot from Player 2
            Player2.balance -= 1
            pot += 1
            
            # second action by Player 1
            print("Player 1 Balance", Player1.balance)
            print("Player 1, do you choose to fold or call")
            print("Press 0 for fold, 1 to call")
            print()
            player1_action2 = int(input())
            
            # execute if Player 1 folds
            if player1_action2 == 0:
                Player2.balance += pot
                print("Player 2 Wins")
                print("Player 1 Card:", Player1.card)
                print("Player 1 Balance:", Player1.balance)
                print()
                print("Player 2 Card:", Player2.card)
                print("Player 2 Balance:", Player2.balance)
                print()
                pot = 0
        
            # execute if Player 1 calls
            if player1_action2 == 1:
            
                # add to the pot from Player 1
                Player1.balance -= 1
                pot += 1
            
                # showdown for the pot
                
                # Player 1 wins
                if showdownValue(Player1.card, community_card) > showdownValue(Player2.card, community_card):
                    Player1.balance += pot
                    print("Player 1 Wins")
                    print("Player 1 Card:", Player1.card)
                    print("Player 1 Balance:", Player1.balance)
                    print()
                    print("Player 2 Card:", Player2.card)
                    print("Player 2 Balance:", Player2.balance)
                    print()
                    pot = 0
                
            
                # Player 2 wins            
                elif showdownValue(Player2.card, community_card) > showdownValue(Player1.card, community_card):
                    Player2.balance += pot
                    print("Player 2 Wins")
                    print("Player 1 Card:", Player1.card)
                    print("Player 1 Balance:", Player1.balance)
                    print()
                    print("Player 2 Card:", Player2.card)
                    print("Player 2 Balance:", Player2.balance)
                    print()
                    pot = 0
                    
                
                # Tie Game
                else:
                    Player1.balance += pot/2
                    Player2.balance += pot/2
                    pot = 0
                    return "Draw"
            
    
    # execute if Player 1 bets
    if player1_action1 == 1:
        
        # add to the pot from Player 1
        Player1.balance -= 1
        pot += 1
        
        # first action by Player 2
        print("Player 2 Balance", Player2.balance)
        print()
        if Player2.state[p2[0] + '1'] > Player2.state[p2[0] + '0']:
            player2_action1 = 1
        elif Player2.state[p2[0] + '1'] < Player2.state[p2[0] + '0']:
            player2_action1 = 0
        else:
            player2_action1 = round(random.random())
            
        p2.append(p2[0] + str(player2_action1))
        
        if player2_action1 == 1:
            print("Player 2 bets")
        else:
            print("Player 2 checks")
        
        # execute if Player 2 folds
        if player2_action1 == 0:
            Player1.balance += pot
            print("Player 1 Wins")
            print("Player 1 Card:", Player1.card)
            print("Player 1 Balance:", Player1.balance)
            print()
            print("Player 2 Card:", Player2.card)
            print("Player 2 Balance:", Player2.balance)
            print()
            pot = 0
        
        # execute if Player 2 calls
        if player2_action1 == 1:
            
            # add to the pot from Player 2
            Player2.balance -= 1
            pot += 1
            
            # showdown for the pot
                
            # Player 1 wins
            if showdownValue(Player1.card, community_card) > showdownValue(Player2.card, community_card):
                Player1.balance += pot
                print("Player 1 Wins")
                print("Player 1 Card:", Player1.card)
                print("Player 1 Balance:", Player1.balance)
                print()
                print("Player 2 Card:", Player2.card)
                print("Player 2 Balance:", Player2.balance)
                print()
                pot = 0
                
            
            # Player 2 wins            
            elif showdownValue(Player2.card, community_card) > showdownValue(Player1.card, community_card):
                Player2.balance += pot
                print("Player 2 Wins")
                print("Player 1 Card:", Player1.card)
                print("Player 1 Balance:", Player1.balance)
                print()
                print("Player 2 Card:", Player2.card)
                print("Player 2 Balance:", Player2.balance)
                print()
                pot = 0
            
            # Tie Game
            else:
                Player1.balance += pot/2
                Player2.balance += pot/2
                pot = 0
                return "Draw"

In [61]:
Player1 = HumanPlayer(1000, '', 0)
Player2 = AIPlayer(1000, '', 0, 0, 0, s2)
for i in range(10):
    print(f"**************** Game {i+1}: ****************")
    LeducPokerPvC(Player1, Player2)
    print( "---------------------------------------------")

**************** Game 1: ****************
Player 1 Card: K
Player 1 Balance: 1000

Player 2 Card not shown
Player 2 Balance: 1000

Community Card: J


Initial bets of 1 made

Player 1 Balance 999
Player 1, do you choose to check or bet
Press 0 for check, 1 to bet



 0


Player 2 Balance 999

Player 2 bets
Player 1 Balance 999
Player 1, do you choose to fold or call
Press 0 for fold, 1 to call



 1


---------------------------------------------
**************** Game 2: ****************
Player 1 Card: Q
Player 1 Balance: 1000.0

Player 2 Card not shown
Player 2 Balance: 1000.0

Community Card: J


Initial bets of 1 made

Player 1 Balance 999.0
Player 1, do you choose to check or bet
Press 0 for check, 1 to bet



...

Player 2 Balance 1004.0

Player 2 bets
Player 1 Balance 994.0
Player 1, do you choose to fold or call
Press 0 for fold, 1 to call



 0


Player 2 Wins
Player 1 Card: J
Player 1 Balance: 994.0

Player 2 Card: K
Player 2 Balance: 1006.0

---------------------------------------------
**************** Game 10: ****************
Player 1 Card: K
Player 1 Balance: 994.0

Player 2 Card not shown
Player 2 Balance: 1006.0

Community Card: K


Initial bets of 1 made

Player 1 Balance 993.0
Player 1, do you choose to check or bet
Press 0 for check, 1 to bet



 1


Player 2 Balance 1005.0

Player 2 bets
Player 2 Wins
Player 1 Card: K
Player 1 Balance: 992.0

Player 2 Card: Q
Player 2 Balance: 1008.0

---------------------------------------------


After playing 10 rounds with the AI going second, the AI took $8 from us.